# Fabric ETL Demo (PySpark)
This notebook ingests CSVs from **/Files/data** in a Fabric Lakehouse, cleans them, and writes curated Delta Tables.


In [ ]:
# Fabric/Databricks-compatible PySpark
from pyspark.sql import functions as F
from pyspark.sql import types as T

# Adjust base path if needed. In Fabric, upload CSVs to Lakehouse Files: /Files/data
base_path = "/lakehouse/default/Files/data"  # In Fabric, mount differs by environment; change as needed.
customers_path = f"{base_path}/customers.csv"
orders_path = f"{base_path}/orders.csv"

df_customers = (spark.read
    .option("header", True)
    .csv(customers_path)
)

df_orders = (spark.read
    .option("header", True)
    .csv(orders_path)
    .withColumn("Qty", F.col("Qty").cast("int"))
    .withColumn("UnitPrice", F.col("UnitPrice").cast("double"))
    .withColumn("OrderDate", F.to_date("OrderDate"))
    .withColumn("LineTotal", F.col("Qty") * F.col("UnitPrice"))
)

df_customers.createOrReplaceTempView("stg_customers")
df_orders.createOrReplaceTempView("stg_orders")
display(df_orders.limit(5))


In [ ]:
# Join and curate a simple star-like table
df_fact = spark.sql('''
SELECT o.OrderID,
       o.OrderDate,
       o.CustomerID,
       c.FirstName,
       c.LastName,
       c.City,
       c.State,
       o.SKU,
       o.Qty,
       o.UnitPrice,
       o.LineTotal
FROM stg_orders o
LEFT JOIN stg_customers c
  ON o.CustomerID = c.CustomerID
''')

display(df_fact.limit(10))


In [ ]:
# Write to Lakehouse Tables (Delta)
catalog_table = "fact_orders"
df_fact.write.mode("overwrite").saveAsTable(catalog_table)
print(f"Wrote table: {catalog_table}")


## Next steps
- Open the **SQL analytics endpoint** for your Lakehouse and query `fact_orders`.
- Connect a **Power BI semantic model** via **Direct Lake**.
- Add measures like `Total Sales = SUM(fact_orders[LineTotal])`.
